<a href="https://colab.research.google.com/github/Veenkatacharan/AI/blob/main/SSCI_Springer_DOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time

input_excel_path = "SSCI_Springer.xlsx"  # path to file
output_csv_path = "ssci_springer_doi_list1.csv"
email_for_crossref = "veenkata2002@gmail.com"

start_year = 1990
end_year = 2025
max_rows_per_query = 100

df = pd.read_excel(input_excel_path)
df["Preferred_ISSN"] = df["ISSN"].fillna(df["eISSN"])
df = df.dropna(subset=["Preferred_ISSN"]).copy()
issn_list = df["Preferred_ISSN"].unique().tolist()

def query_crossref_for_issn(issn, year):
    url = f"https://api.crossref.org/journals/{issn}/works"
    params = {
        "filter": f"from-pub-date:{year}-01-01,until-pub-date:{year}-12-31",
        "rows": max_rows_per_query,
        "mailto": email_for_crossref
    }
    response = requests.get(url, params=params, timeout=20)
    response.raise_for_status()
    return response.json().get("message", {}).get("items", [])

results = []
for issn in issn_list:
    for year in range(start_year, end_year + 1):
        try:
            print(f"Querying ISSN {issn} for year {year}...")
            items = query_crossref_for_issn(issn, year)
            for item in items:
                results.append({
                    "ISSN": issn,
                    "Year": year,
                    "DOI": item.get("DOI", ""),
                    "Title": item.get("title", [""])[0],
                    "Journal": item.get("container-title", [""])[0],
                    "Publisher": item.get("publisher", "")
                })
            time.sleep(1)
        except Exception as e:
            print(f"Failed for ISSN {issn} ({year}): {e}")
            continue

df_out = pd.DataFrame(results)
df_out.to_csv(output_csv_path, index=False)
print(f"Saved {len(df_out)} records to {output_csv_path}")